## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [3]:
# TODO: Load environment variables
# load_dotenv()
import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv('OPENAI_API_KEY') is not None
assert os.getenv('TAVILY_API_KEY') is not None

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### VectorDB Instance

In [4]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
# chroma_client = chromadb.PersistentClient(path="chromadb")
import chromadb

chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [5]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
# embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

embeddings_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, api_base="https://openai.vocareum.com/v1")


In [6]:
# TODO: Create a collection
# Choose any name you want
# collection = chroma_client.create_collection(
#    name="udaplay",
#    embedding_function=embedding_fn
#)
try:
    chroma_client.delete_collection(name="udaplay_games")
except:
    print(f"ℹ No existing collection to delete") 
  
collection = chroma_client.create_collection(name="udaplay_games", embedding_function=embeddings_fn)

ℹ No existing collection to delete


### Add documents

In [7]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

Search

In [9]:
test_queries = [
    "racing games from the 90s",
    "RPG games with turn-based combat",
    "action games for PlayStation"
]

for query in test_queries:
    
    results = collection.query(
        query_texts=[query],
        n_results=3
    )

    print(f" {results} \n")


 {'ids': [['001', '003', '012']], 'embeddings': None, 'documents': [['[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.', '[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.', '[Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'Name': 'Gran Turismo', 'YearOfRelease': 1997, 'Genre': 'Racing', 'Platform': 'PlayStation 1', 'Publisher': 'Sony Computer Entertainment', 'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'}, {'Genre': 'Racing', 'Description': 'A comprehensive racing simulator featuring a vast selection of v